In [1]:
# suppress annoying pandas warnings
from contextlib import contextmanager
import logging
import warnings

import pandas as pd

warnings.simplefilter('ignore')

logging.basicConfig()
logger = logging.getLogger()

@contextmanager
def debug_log():
    logger.setLevel(logging.DEBUG)
    yield
    logger.setLevel(logging.WARNING)

In [2]:
DRAFT_YEARS = [2014, 2015, 2016, 2017, 2018]
DRAFT_URL = "https://www.basketball-reference.com/draft/NBA_{year}.html"


def draft_for_year(year):
    url = DRAFT_URL.format(year=year)
    logging.debug("Getting data for {0}".format(url))
    return pd.read_html(url, header=1, skiprows=[32, 33])[0]


def get_draft_data():
    
    frames = []
    for year in DRAFT_YEARS:
        draft_data = draft_for_year(year)
        draft_data['Draft_Year'] = year
        frames.append(draft_data)
    
    return pd.concat(frames)

In [13]:
PLAYER_YEARS = [y + 1 for y in DRAFT_YEARS]
PLAYER_URL = "https://www.basketball-reference.com/leagues/NBA_{year}_per_game.html"

# Fix inconcistencies between players and drafted players listings
MANUAL_PLAYER_MAPPINGS = {
    'Timothe Luwawu-Cabarrot': 'Timothe Luwawu',
    'Taurean Waller-Prince': 'Taurean Prince',
    "DeAndre' Bembry": 'DeAndre Bembry',
    "Johnny O'Bryant": "Johnny O'Bryant III",
    "Glenn Robinson": "Glenn Robinson III",
    "Devyn Marble": "Roy Devyn Marble",
    "Larry Nance": "Larry Nance Jr.",
    "Joe Young": "Joseph Young",
    'Walter Tavares': "Edy Tavares"
}

def player_data_for_year(year):
    url = PLAYER_URL.format(year=year)
    logging.debug("Getting data for {0}".format(url))
    data = pd.read_html(url)[0]
    
    # 1) remove header rows
    data = data[data.Rk != 'Rk']
    
    # 2) cast to the correct type
    non_numeric_cols = set(['Player', 'Pos', 'Tm'])
    for col in data.columns:
        if col in non_numeric_cols:
            continue
        data.loc[:, col] = pd.to_numeric(data[col], errors='downcast')  
    return data


def get_player_raw_data():
    frames = []
    for year in PLAYER_YEARS:
        player_data = player_data_for_year(year)
        player_data['Year'] = year
        frames.append(player_data)
    
    data = pd.concat(frames)
    
    # fix inconsistent naming between draft and player listings
    data['Player'] = data['Player'].apply(lambda pn: MANUAL_PLAYER_MAPPINGS.get(pn, pn))
    return data


def min_year_and_max_games_played_for_team(data):
    
    players_by_min_years = data[['Player', 'Year']].groupby('Player').min().reset_index()
    players_min_years_only = pd.merge(data, players_by_min_years, on=['Player', 'Year'], how='inner')
    
    # find the team which the player played the most games for
    no_tm = players_min_years_only[players_min_years_only.Tm != 'TOT']
    player_groupby = no_tm.groupby(by='Player')
    players_single_team = player_groupby[['G', 'Tm', 'Year']].max().reset_index()
    return players_single_team

In [4]:
def merge_draft_year_with_player_data(draft_data, player_data):
    
    # remove unwanted columns before the merge
    draft_data.drop(columns=['Tm', 'Rk'], inplace=True)
    player_data.drop(columns=['G'], inplace=True)
    
    return pd.merge(player_data, draft_data, how='inner', on='Player')

In [14]:
with debug_log():
    draft_data = get_draft_data()
    player_raw_data = get_player_raw_data()
    players_by_min_year_max_games = min_year_and_max_games_played_for_team(player_raw_data)
    
    merged = merge_draft_year_with_player_data(draft_data, players_by_min_year_max_games)
    merged['WS/Year'] = merged['WS'] / merged['Yrs']

DEBUG:root:Getting data for https://www.basketball-reference.com/draft/NBA_2014.html
DEBUG:root:Getting data for https://www.basketball-reference.com/draft/NBA_2015.html
DEBUG:root:Getting data for https://www.basketball-reference.com/draft/NBA_2016.html
DEBUG:root:Getting data for https://www.basketball-reference.com/draft/NBA_2017.html
DEBUG:root:Getting data for https://www.basketball-reference.com/draft/NBA_2018.html
DEBUG:root:Getting data for https://www.basketball-reference.com/leagues/NBA_2015_per_game.html
DEBUG:root:Getting data for https://www.basketball-reference.com/leagues/NBA_2016_per_game.html
DEBUG:root:Getting data for https://www.basketball-reference.com/leagues/NBA_2017_per_game.html
DEBUG:root:Getting data for https://www.basketball-reference.com/leagues/NBA_2018_per_game.html
DEBUG:root:Getting data for https://www.basketball-reference.com/leagues/NBA_2019_per_game.html


In [32]:
# import matplotlib.pyplot as plt
# %matplotlib inline


# merged.plot(kind='scatter', x='Pk', y='VORP', figsize=(12, 5))

In [25]:
draft_data.columns

Index(['Pk', 'Player', 'College', 'Yrs', 'G', 'MP', 'PTS', 'TRB', 'AST', 'FG%',
       '3P%', 'FT%', 'MP.1', 'PTS.1', 'TRB.1', 'AST.1', 'WS', 'WS/48', 'BPM',
       'VORP', 'Draft_Year'],
      dtype='object')

In [23]:
merged[merged.Tm == 'DEN'].sort_values(by='Pk')

,Player,Tm,Year,Pk,College,Yrs,G,MP,PTS,TRB,...,MP.1,PTS.1,TRB.1,AST.1,WS,WS/48,BPM,VORP,Draft_Year,WS/Year
75,Emmanuel Mudiay,DEN,2016,7,NaN,4.0,226.0,5755.0,2602.0,682.0,...,25.5,11.5,3.0,4.2,-1.1,-0.009,-3.8,-2.6,2015,-0.275000
105,Jamal Murray,DEN,2017,7,University of Kentucky,3.0,214.0,6036.0,3094.0,727.0,...,28.2,14.5,3.4,3.3,9.7,0.077,-1.1,1.3,2016,3.233333
138,Juan Hernangomez,DEN,2017,15,NaN,3.0,141.0,2336.0,780.0,485.0,...,16.6,5.5,3.4,0.6,5.1,0.106,-1.0,0.6,2016,1.700000
145,Jusuf Nurkic,DEN,2015,16,NaN,5.0,294.0,6658.0,3324.0,2303.0,...,22.6,11.3,7.8,1.8,14.4,0.104,1.0,5.1,2014,2.880000
81,Gary Harris,DEN,2015,19,Michigan State University,5.0,287.0,8200.0,3614.0,735.0,...,28.6,12.6,2.6,2.2,15.1,0.089,0.1,4.3,2014,3.020000
170,Malik Beasley,DEN,2017,19,Florida State University,3.0,141.0,2119.0,961.0,231.0,...,15.0,6.8,1.6,0.8,4.4,0.100,-1.6,0.2,2016,1.466667
243,Tyler Lydon,DEN,2018,24,Syracuse University,2.0,22.0,89.0,21.0,16.0,...,4.0,1.0,0.7,0.2,0.1,0.065,-3.2,0.0,2017,0.050000
111,Jarred Vanderbilt,DEN,2019,41,University of Kentucky,1.0,3.0,14.0,1.0,11.0,...,4.7,0.3,3.7,0.3,0.0,-0.049,-5.9,0.0,2018,0.000000
193,Nikola Jokic,DEN,2016,41,NaN,4.0,284.0,7981.0,4544.0,2676.0,...,28.1,16.0,9.4,5.1,35.8,0.215,7.5,19.1,2014,8.950000
187,Monte Morris,DEN,2018,51,Iowa State University,2.0,60.0,1449.0,626.0,153.0,...,24.2,10.4,2.6,3.8,4.9,0.161,0.5,0.9,2017,2.450000


In [18]:
merged['WS/Year'] = merged['WS'] / merged['Yrs']
merged.to_csv("/Users/jimmyzhang/Documents/draft_analysis.csv", index=False)

In [17]:
# ==== utility functions for cleaning ====


def get_unmerged_draft_players(draft_data, merged):
    return draft_data[(~draft_data.Player.isin(merged.Player))][['Player', 'G', 'Draft_Year']].dropna()

get_unmerged_draft_players(draft_data, merged)

,Player,G,Draft_Year
42,Walter Tavares,13.0,2014


In [ ]:
player_advanced_2019 = player_data_for_year()

In [48]:

def advanced_player_data_for_year():
    url = 'https://www.basketball-reference.com/leagues/NBA_2019_advanced.html'
    logging.debug("Getting data for {0}".format(url))
    data = pd.read_html(url)[0]
    
    # 1) remove header rows
    data = data[data.Rk != 'Rk']
    
    # 2) cast to the correct type
    non_numeric_cols = set(['Player', 'Pos', 'Tm'])
    for col in data.columns:
        if col in non_numeric_cols:
            continue
        data.loc[:, col] = pd.to_numeric(data[col], errors='downcast')
    
    return data.drop(columns=['Unnamed: 19', 'Unnamed: 24'])
        
#     # take only the total stat
#     no_tm = data[data.Tm != 'TOT']
#     player_groupby = data.groupby(by='Player')
#     return players_single_team.drop()
    
adv_data = advanced_player_data_for_year()


In [108]:
den_pks = set(merged[merged['Tm'] == 'DEN'].Pk.values)
ph_pks = set(merged[merged['Tm'] == 'PHO'].Pk.values)



merged[merged.apply(lambda x: x.Pk in den_pks or x.Pk in ph_pks, axis=1)]

,Player,Tm,Year,Pk,College,Yrs,G,MP,PTS,TRB,...,MP.1,PTS.1,TRB.1,AST.1,WS,WS/48,BPM,VORP,Draft_Year,WS/Year
0,A.J. Hammons,DAL,2017,46,Purdue University,1.0,22.0,163.0,48.0,36.0,...,7.4,2.2,1.6,0.2,0.0,-0.001,-5.6,-0.1,2016,0.000000
1,Aaron Gordon,ORL,2015,4,University of Arizona,5.0,318.0,8708.0,3870.0,1940.0,...,27.4,12.2,6.1,2.0,16.4,0.090,0.1,4.7,2014,3.280000
3,Abdel Nader,BOS,2018,58,Iowa State University,2.0,86.0,907.0,293.0,138.0,...,10.5,3.4,1.6,0.4,0.4,0.024,-5.7,-0.8,2016,0.200000
4,Adreian Payne,MIN,2015,15,Michigan State University,4.0,107.0,1403.0,429.0,315.0,...,13.1,4.0,2.9,0.6,-0.6,-0.019,-5.5,-1.2,2014,-0.150000
5,Alec Peters,PHO,2018,54,Valparaiso University,1.0,20.0,225.0,82.0,37.0,...,11.3,4.1,1.9,0.6,0.3,0.055,-3.3,-0.1,2017,0.300000
8,Andrew Wiggins,MIN,2015,1,University of Kansas,5.0,378.0,13613.0,7360.0,1596.0,...,36.0,19.5,4.2,2.1,12.9,0.046,-2.5,-1.7,2014,2.580000
9,Anfernee Simons,POR,2019,24,NaN,1.0,12.0,60.0,15.0,7.0,...,5.0,1.3,0.6,0.3,-0.4,-0.296,-16.8,-0.2,2018,-0.400000
11,Anthony Brown,LAL,2016,34,Stanford University,3.0,41.0,762.0,161.0,103.0,...,18.6,3.9,2.5,0.7,-0.2,-0.016,-4.8,-0.5,2015,-0.066667
12,Bam Adebayo,MIA,2018,14,University of Kentucky,2.0,125.0,2604.0,916.0,755.0,...,20.8,7.3,6.0,1.7,8.2,0.152,1.2,2.1,2017,4.100000
13,Ben Bentil,DAL,2017,51,Providence College,1.0,3.0,10.0,0.0,2.0,...,3.3,0.0,0.7,0.0,-0.1,-0.473,-26.9,-0.1,2016,-0.100000


In [76]:
drop_columns = ['PTS', 'G', 'MP', 'TRB', 'AST', 
                'FG%','3P%', 'FT%', 'MP.1', 'PTS.1', 'TRB.1', 'AST.1', 
                'WS', 'WS/48', 'BPM', 'VORP', 'Year', 'WS/Year']

copy = merged.drop(columns=drop_columns)
copy.rename(columns={'Tm': 'Draft Team'}, inplace=True)
merged_2019 = pd.merge(copy, adv_data, how='inner', on="Player")

In [90]:
merged_2019[merged_2019.MP > 500].sort_values(by='PER', ascending=False)

,Player,Draft Team,Pk,College,Yrs,Draft_Year,Rk,Pos,Age,Tm,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
177,Nikola Jokic,DEN,41,NaN,4.0,2014,256,C,23,DEN,...,15.6,27.7,5.9,2.8,8.7,0.236,6.3,3.4,9.7,5.2
134,Karl-Anthony Towns,MIN,1,University of Kentucky,4.0,2015,453,C,23,MIN,...,13.4,27.4,5.3,2.7,8.0,0.203,4.5,2.4,6.9,4.2
111,Joel Embiid,PHI,3,University of Kansas,3.0,2014,149,C,24,PHI,...,13.5,33.2,4.1,3.1,7.3,0.192,1.4,1.9,3.3,2.4
30,Clint Capela,HOU,25,NaN,5.0,2014,86,C,24,HOU,...,10.1,19.3,5.0,1.5,6.4,0.215,1.5,1.0,2.6,1.6
171,Montrezl Harrell,HOU,32,University of Louisville,4.0,2015,199,C,25,LAC,...,10.2,22.4,4.6,1.9,6.5,0.200,2.8,1.8,4.6,2.6
132,Jusuf Nurkic,DEN,16,NaN,5.0,2014,359,C,24,POR,...,14.4,24.4,3.1,2.6,5.7,0.180,1.2,3.8,5.1,2.7
56,Domantas Sabonis,OKC,11,Gonzaga University,3.0,2016,410,C,22,IND,...,16.7,23.0,3.5,2.7,6.3,0.217,1.8,2.4,4.1,2.2
188,Richaun Holmes,PHI,37,Bowling Green State University,4.0,2015,225,C,25,PHO,...,10.9,17.6,2.6,0.8,3.4,0.186,0.6,1.7,2.2,0.9
123,Julius Randle,LAL,7,University of Kentucky,5.0,2014,393,C,24,NOP,...,13.8,26.7,3.5,1.4,4.9,0.153,1.9,-0.2,1.8,1.5
112,John Collins,ATL,19,Wake Forest University,2.0,2017,104,PF,21,ATL,...,12.2,23.5,3.4,0.6,4.0,0.152,2.9,-1.6,1.2,1.0


## import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


dims = (16, 9)
fig, ax = plt.subplots(figsize=dims)

team = 'DEN'
sub = merged_2019[merged_2019.MP > 500]
sub['is_team'] = sub['Tm'].apply(lambda x: x == team)

sns.scatterplot(ax=ax, data=sub, x='Pk', y='VORP', hue='is_team', legend=False)